In [ ]:
import pandas as pd
import mysql.connector

def upload_csv_to_sql(csv_file, table_name, connection_config):
    # Read CSV file into DataFrame
    df = pd.read_csv(csv_file)
    
    # Connect to MySQL server
    try:
        conn = mysql.connector.connect(**connection_config)
        cursor = conn.cursor()

        # Check if the table already exists
        cursor.execute(f"SHOW TABLES LIKE '{table_name}'")
        table_exists = cursor.fetchone()

        # If the table doesn't exist, create it
        if not table_exists:
            # Create table with columns based on DataFrame columns
            columns = ', '.join(f"{col} VARCHAR(255)" for col in df.columns)
            create_table_query = f"CREATE TABLE {table_name} ({columns})"
            cursor.execute(create_table_query)
            print(f"Created table '{table_name}'.")

        # Insert data into the table
        for _, row in df.iterrows():
            values = ', '.join(f"'{value}'" for value in row.values)
            insert_query = f"INSERT INTO {table_name} VALUES ({values})"
            cursor.execute(insert_query)

        conn.commit()
        print(f"Appended data from '{csv_file}' to '{table_name}'.")

    except mysql.connector.Error as e:
        print("Error occurred while uploading data to MySQL:", e)

    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()

if __name__ == "__main__":
    csv_file = input("Enter the path to the CSV file: ")
    table_name = input("Enter the name of the SQL table: ")
    host = input("Enter the host (default is 'localhost'): ") or "localhost"
    port = input("Enter the port (default is '3306'): ") or "3306"
    user = input("Enter the user: ")
    password = input("Enter the password: ")
    database = input("Enter the database name: ")

    # Define connection configuration
    connection_config = {
        'host': host,
        'port': port,
        'user': user,
        'password': password,
        'database': database
    }

    # Upload CSV to SQL table
    upload_csv_to_sql(csv_file, table_name, connection_config)
